# Prediction

inputs are train.csv, prediction.csv, output_dir, target 
where target is one of the six outcomes: gpa|grit|materialHardship|eviction|layoff|jobTraining


In [ ]:
'''
* summary: regression and classification
* input: bg, tr, pr
* output: predictions for NA values in pr
'''
%matplotlib inline
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from theano import shared
import theano.tensor as T
import pymc3 as pm
import theano
import sys

train = "/tigress/BEE/projects/rufragfam/ai4all/data/train.csv"
prediction = "/tigress/BEE/projects/rufragfam/ai4all/data/prediction.csv"
output_dir = "/tigress/BEE/projects/rufragfam/ai4all/output"

# define outcomes
disout = ['eviction','layoff','jobTraining']
conout = ['gpa','grit','materialHardship']
outcome = conout[0]
allout = disout+conout

# np.random.seed(int(1234)) # for reproducibility

# read in data
bg = pd.read_csv(output_dir+'/'+outcome+'_fselected_bg.csv',index_col=0)

nb_samples = bg.shape[0]

tr = pd.read_csv(train, low_memory=False)
tr = tr.set_index('challengeID')
    
pr = pd.read_csv(prediction, low_memory=False)
pr = pr.set_index('challengeID')

# by default we set all values in the prediction set to a number we wont see (-999)
pr[:]=-999
# then we add in the training samples
pr.update(tr)
#pr=pr.head(100)
#bg=bg.head(100)
tout = np.asarray(pr[outcome])

print "updated the predicted data frame with training samples"

In [ ]:
'''
* Input: prefixes and prefix to wave/person mappings
* Output: var_to_cols mapping of variable base to columns; 
* all variable roots are stored in the set var_roots
* you can reconstruct information about a variable with splitVar(colval) 
*         which returns a tuple (is constructed?,prefix,variable base, wave)
* to retrieve all variables associated with variable base varbase bg[var_to_cols[varbase]]
'''

# figure out which covariates are normally distributed and which are bernoulli
norm_cols=[]
bi_cols=[]
norm_cols_idx=[]
bi_cols_idx=[]
idx = 0
for col_name in bg:
    bg[col_name]
    col=bg[col_name]
    if col.min()>=0 and col.max()-col.min()<=1:
        bi_cols.append(col_name)
        bi_cols_idx.append(idx)
    else:
        norm_cols.append(col_name)
        norm_cols_idx.append(idx)
    idx+=1

# set up mapping from variable stem, to (variable ids, wave no.)
all_prefix=["m1","m2","m3","m4","m5","f1","f2","f3","f4","f5","hv3","hv4","hv5","p5","k5",
            "ffcc","kind","t5","emp3","emp4","emp5","c3","c4","c5","n5","o5","mf4"]
prefix_to_wave_personID = {"m1":(1,"mother"),"m2":(2,"mother"),"m3":(3,"mother"),"m4":(4,"mother"),"m5":(5,"mother"),"f1":(1,"father"),
                           "f2":(2,"father"),"f3":(3,"father"),"f4":(4,"father"),"f5":(5,"father"),"hv3":(3,"home_visit"),
                           "hv4":(4,"home_visit"),"hv5":(5,"home_visit"),"p5":(5,"primary_caregiver"),"k5":(5,"child"),
                           "ffcc":(3,"child_care_provider"),"kind":(4,"kindergarten_teacher"),"t5":(5,"teacher"),"emp3":(3,"emp"),
                           "emp4":(4,"emp"),"emp5":(5,"emp"),"c3":(3,"c"),"c4":(4,"c"),"c5":(4,"c"),"n5":(5,"n"),"o5":(5,"o"),
                           "mf4":(4,'mother-father')}

def splitVar(varname):
    con_pre_varbase_wave = []
    newcolval = varname
    if varname[0]=='c':
        con_pre_varbase_wave.append(True)
        newcolval=varname[1:]
    else:
        con_pre_varbase_wave.append(False)
    prefixes_for_this_col = []
    varbase = newcolval
    for prefix in all_prefix:
        if newcolval.startswith(prefix):
            prefixes_for_this_col.append(prefix)
    if len(prefixes_for_this_col)==0:             
        print "could not find prefix for variable " + str(newcolval)
        con_pre_varbase_wave.append('')
        con_pre_varbase_wave.append(varbase)
    elif len(prefixes_for_this_col)>1:
        print "too many matching prefixes for variable " + str(newcolval)
        con_pre_varbase_wave.append('')
        con_pre_varbase_wave.append(varbase)
    else:
        con_pre_varbase_wave.append(prefixes_for_this_col[0])
        if con_pre_varbase_wave[0]:
            varbase = varname[len('c'+prefixes_for_this_col[0]):]
        else:
            varbase = varname[len(prefixes_for_this_col[0]):]
        con_pre_varbase_wave.append(varbase)
    if con_pre_varbase_wave[1]!='':
        con_pre_varbase_wave.append(prefix_to_wave_personID[con_pre_varbase_wave[1]][0])
    else:
        con_pre_varbase_wave.append('')
    return con_pre_varbase_wave
    
# construct variable roots and a map to the variables across time
# note that the termporal component is not currently used in the GLM
var_to_cols = {}
var_roots = set()
for colval in bg:
    con_pre_varbase_wave = splitVar(colval)
    var = con_pre_varbase_wave[2]
    if var not in var_to_cols:
        var_to_cols[var]=[]
    var_to_cols[var].append(colval)
    var_roots.add(splitVar(colval)[2])
    
print "constructed variable to columns mapping across waves"

In [ ]:
# basic regularized regression
# get training data for a response variable ('gpa', 'grit', 'materialHardship', 
# 'eviction', 'layoff', or 'jobTraining) in typical X, y format
def get_training_data(response):
    y = tr[response]
    y = y[y.notnull()]

    train_ids = y.index.tolist()
    y = y.values

    X = bg.ix[train_ids].as_matrix()
    
    return X, y, train_ids

# get testing data for a response variable ('gpa', 'grit', 'materialHardship', 
# 'eviction', 'layoff', or 'jobTraining) -- only X
def get_testing_data(response):
    y = tr[response]
    y = y[y.notnull()]

    train_ids = y.index.tolist()
    test_ids = [i for i in bg.index.tolist() if i not in train_ids]
    test_ids = sorted(test_ids)
    X = bg.ix[test_ids].as_matrix()
    
    return X, test_ids

def split_data(X, y, split=0.1):
    assert X.shape[0] == y.shape[0]
    indices = range(0, X.shape[0])
    np.random.shuffle(indices)

    pivot = int(X.shape[0] * split)
    
    major_indices = indices[pivot:]
    minor_indices = indices[:pivot]
    
    X_major = X[major_indices, :]
    X_minor = X[minor_indices, :]
    y_major = y[major_indices]
    y_minor = y[minor_indices]
    
    return (X_major, y_major), (X_minor, y_minor)
    
'''
* summary: provides functions for performing lasso
'''

def predict(response, model_func, nb_trials):
    is_binary = response in ['eviction', 'layoff', 'jobTraining']

    X_train_all, y_train_all, train_ids = get_training_data(response)

    scores = []
    for i in range(nb_trials):
        (X_train, y_train), (X_val, y_val) = \
            split_data(X_train_all, y_train_all, split=0.5)

        # validation phase (estimate performance)
        model = model_func()
        model.fit(X_train, y_train)

        if is_binary:
            y_preds = model.predict_proba(X_val)[:, 1]
        else:
            y_preds = model.predict(X_val)

        if is_binary: 
            loss = np.mean((y_preds - y_val) ** 2) # brier
        else:   
            loss = np.mean((y_preds - y_val) ** 2) # mse
        scores.append(loss)
        
        print('Trial {}/{} done'.format(i + 1, nb_trials))
        
    if is_binary:
        print("{} for {} / avg Brier loss = {:.4}"
              .format(model_func.__name__, response, np.mean(scores)))
    else:
        print("{} for {} / avg mse = {:.4}"
            .format(model_func.__name__, response, np.mean(scores)))

    # model serving phase (on actual test data)
    model = model_func()
    model.fit(X_train_all, y_train_all)

    X_test, test_ids = get_testing_data(response)
    
    assert X_test.shape[0] + X_train_all.shape[0] == bg.shape[0]
    assert X_test.shape[1] == X_train.shape[1]
    
    if is_binary:
        y_preds = model.predict_proba(X_test)[:,1]
    else:
        y_preds = model.predict(X_test)
    
    out = []
    
    train_dict = dict(zip(train_ids, y_train_all))
    test_dict = dict(zip(test_ids, y_preds))
    
    assert len(train_dict) + len(test_dict) == nb_samples
    
    for i in range(1, nb_samples+1):
        if i in train_dict:
            out.append(train_dict[i])
        else:
            assert i in test_dict
            out.append(test_dict[i])
            
    return out


def lasso_predict(response, nb_trials=5):
    from sklearn import linear_model
    return predict(response, linear_model.Lasso, nb_trials=nb_trials)

def logit_predict(response, nb_trials=5):
    from sklearn import linear_model
    return predict(response, linear_model.LogisticRegression, nb_trials=nb_trials)


PRODUCTION = True

if PRODUCTION:
    pr['eviction'] = logit_predict('eviction', nb_trials=1)
    pr['layoff'] = logit_predict('layoff', nb_trials=1)
    pr['jobTraining'] = logit_predict('jobTraining', nb_trials=1)
    pr['gpa'] = lasso_predict('gpa', nb_trials=1)
    pr['grit'] = lasso_predict('grit', nb_trials=1)
    pr['materialHardship'] = lasso_predict('materialHardship', nb_trials=1)
else:
    pr['eviction'] = logit_predict('eviction')
    pr['layoff'] = logit_predict('layoff')
    pr['jobTraining'] = logit_predict('jobTraining')
    pr['gpa'] = lasso_predict('gpa')
    pr['grit'] = lasso_predict('grit')
    pr['materialHardship'] = lasso_predict('materialHardship')
    
print "output recorded"
pr.to_csv(output_dir+'/regression_prediction.csv', index=False)